# 최성보(최소성취수준 보장지도) 위험군 조기 예측 — 분석 노트북

본 노트북은 **정보·컴퓨터 교과**에서 학기 중간 시점의 제한된 데이터로
**최소성취수준 보장지도(최성보) 대상 위험군**을 사전에 선별하기 위한 분석/모델링 파이프라인의 1단계(베이스라인)를 다룹니다.

## 분석 목표
- 학습 행동/평가/출결 지표가 성취도(성취율)와 어떤 관계를 가지는지 탐색(EDA)
- 학습행동 패턴 기반으로 학습자 유형을 군집화(K-Means)
- **예방지도 대상(후보군)** 선별에 활용 가능한 기준 후보를 도출

## 비고
- GitHub에는 **더미 데이터**만 포함합니다.
- 실제 적용 단계에서는 **학기 중간 CSV 업로드 데이터**를 입력으로 받아 최성보 위험군을 예측하는 모델로 확장합니다.

In [2]:
import sys
from pathlib import Path

# 현재 노트북 위치 기준으로 프로젝트 루트 추가
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)

PROJECT_ROOT: d:\workSpace\edutech-learning-analysis


In [4]:
from src.preprocessing import load_csv, preprocess_pipeline

df = load_csv("../data/dummy/dummy_midterm_like.csv")
dfp = preprocess_pipeline(df, add_labels=True)
dfp[["final_score","achievement_rate","at_risk"]].head()

,final_score,achievement_rate,at_risk
0,NaN,50.6,0
1,NaN,52.6,0
2,NaN,60.6,0
3,NaN,58.2,0
4,NaN,31.2,1
